# ASA: International Data Quest: The Price of Progress

## Executive Summary

# Data Management

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt

# ================================
# 1️⃣ LOAD & BERSIHKAN HDI
# ================================
df_hdi = pd.read_excel("./External-Dataset/HDI.xlsx")
# df_hdi = pd.read_excel("HDI.xlsx")

df_hdi.columns = df_hdi.columns.str.strip()
df_hdi = df_hdi[['Country', 'Human Development Index (HDI)']]
df_hdi = df_hdi.rename(columns={
    'Country': 'Country Name',
    'Human Development Index (HDI)': 'HDI_2023'
})
df_hdi['Country Name'] = df_hdi['Country Name'].str.strip()

# ================================
# 2️⃣ LOAD & SAMAKAN NAMA NEGARA
# ================================
df = pd.read_csv("./Dataset/WDICSV-rev.csv")
# df = pd.read_csv("WDICSV-rev.csv")

rename_map = {
    'Bahamas': 'Bahamas, The',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Congo': 'Congo, Rep.',
    'Congo (Democratic Republic of the)': 'Congo, Dem. Rep.',
    "Côte d'Ivoire": "Cote d'Ivoire",
    'Egypt': 'Egypt, Arab Rep.',
    'Eswatini (Kingdom of)': 'Eswatini',
    'Gambia': 'Gambia, The',
    'Hong Kong, China (SAR)': 'Hong Kong SAR, China',
    'Iran (Islamic Republic of)': 'Iran, Islamic Rep.',
    "Korea (Democratic People's Rep. of)": 'Korea, Dem. People’s Rep.',
    'Korea (Republic of)': 'Korea, Rep.',
    'Kyrgyzstan': 'Kyrgyz Republic',
    "Lao People's Democratic Republic": 'Lao PDR',
    'Micronesia (Federated States of)': 'Micronesia, Fed. Sts.',
    'Moldova (Republic of)': 'Moldova',
    'Palestine, State of': 'West Bank and Gaza',
    'Saint Kitts and Nevis': 'St. Kitts and Nevis',
    'Saint Lucia': 'St. Lucia',
    'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
    'Korea, Dem. People’s Rep.': "Korea, Dem. People's Rep.",
    'Slovakia': 'Slovak Republic',
    'Tanzania (United Republic of)': 'Tanzania',
    'Türkiye': 'Turkiye',
    'Venezuela (Bolivarian Republic of)': 'Venezuela, RB',
    'Yemen': 'Yemen, Rep.'
}
df_hdi['Country Name'] = df_hdi['Country Name'].replace(rename_map)

# ================================
# 3️⃣ PILIH INDIKATOR & HITUNG RATA-RATA 5 TAHUN
# ================================
indicators = {
    "NY.GDP.PCAP.KD": "GDP_per_capita",
    "SH.STA.SUIC.P5": "Suicide_rate"
}
years = ['2018', '2019', '2020', '2021', '2022']

df_sel = df[df["Indicator Code"].isin(indicators.keys())].copy()

# Hitung mean dan coverage (berapa persen tahun yang punya data)
df_sel["mean_5yr"] = df_sel[years].mean(axis=1, skipna=True)
df_sel["data_coverage"] = df_sel[years].notna().sum(axis=1) / len(years)

df_mean = df_sel[["Country Name", "Indicator Code", "mean_5yr", "data_coverage"]].drop_duplicates()

df_pivot = df_mean.pivot(index="Country Name", columns="Indicator Code", values="mean_5yr").reset_index()
df_pivot = df_pivot.rename(columns=indicators)

df_coverage = df_mean.pivot(index="Country Name", columns="Indicator Code", values="data_coverage").reset_index()
df_coverage = df_coverage.rename(columns={code: indicators[code] + "_coverage" for code in indicators})

# ================================
# 4️⃣ GABUNGKAN SEMUA DATA
# ================================
df_merge_mean = (
    df_pivot
    .merge(df_hdi, on="Country Name", how="left")
    .merge(df_coverage, on="Country Name", how="left")
)

# Interpolasi nilai yang hilang untuk inklusivitas
df_merge_mean[['GDP_per_capita', 'Suicide_rate', 'HDI_2023']] = df_merge_mean[
    ['GDP_per_capita', 'Suicide_rate', 'HDI_2023']
].interpolate(method='linear', limit_direction='both')

# Tambahkan flag etis: data reliability
df_merge_mean["Low_data_quality_flag"] = np.where(
    (df_merge_mean["GDP_per_capita_coverage"] < 0.6) |
    (df_merge_mean["Suicide_rate_coverage"] < 0.6),
    "⚠️ Low data reliability",
    "✅ Sufficient data"
)

print("\n✅ Data akhir siap dipakai secara etis & transparan:")
print(df_merge_mean.head())

# ================================
# 5️⃣ ANALISIS KORELASI
# ================================
for col in ['HDI_2023', 'GDP_per_capita', 'Suicide_rate']:
    df_merge_mean[col] = pd.to_numeric(df_merge_mean[col], errors='coerce')

# Hapus baris yang punya NaN atau inf di kolom penting
df_corr = df_merge_mean.replace([np.inf, -np.inf], np.nan).dropna(
    subset=['HDI_2023', 'GDP_per_capita', 'Suicide_rate']
)

# Sekarang aman untuk pearson
corr_matrix = df_corr[['HDI_2023', 'GDP_per_capita', 'Suicide_rate']].corr(method='pearson')

r_hdi_suicide, p_hdi_suicide = pearsonr(df_corr['HDI_2023'], df_corr['Suicide_rate'])
r_gdp_suicide, p_gdp_suicide = pearsonr(df_corr['GDP_per_capita'], df_corr['Suicide_rate'])

print("\n📊 Korelasi Matrix:")
print(corr_matrix.round(3))
print(f"\nHDI ↔ Suicide Rate: r = {r_hdi_suicide:.3f}, p = {p_hdi_suicide:.4f}")
print(f"GDP ↔ Suicide Rate: r = {r_gdp_suicide:.3f}, p = {p_gdp_suicide:.4f}")

# Catatan kecil tambahan (opsional)
print(f"\nJumlah negara valid dalam analisis korelasi: {len(df_corr)}")

# ================================
# 6️⃣ CATATAN ETIS
# ================================
print("\n📘 Catatan Etis:")
print("Kami tidak menghapus seluruh observasi dengan data hilang.")
print("Sebaliknya, kami melakukan interpolasi dan memberi penanda 'Low data reliability'")
print("untuk negara dengan cakupan data <60%, agar analisis tetap inklusif sesuai prinsip ASA 2.2.")


✅ Data akhir siap dipakai secara etis & transparan:
                  Country Name  GDP_per_capita  Suicide_rate HDI_2023  \
0                  Afghanistan      485.022544      3.545000    0.496   
1  Africa Eastern and Southern     1429.887639      8.082190      NaN   
2   Africa Western and Central     1806.725660      5.590313      NaN   
3                      Albania     4702.593151      3.430000     0.81   
4                      Algeria     4550.913354      2.002500    0.763   

   GDP_per_capita_coverage  Suicide_rate_coverage Low_data_quality_flag  
0                      1.0                    0.8     ✅ Sufficient data  
1                      1.0                    0.8     ✅ Sufficient data  
2                      1.0                    0.8     ✅ Sufficient data  
3                      1.0                    0.8     ✅ Sufficient data  
4                      1.0                    0.8     ✅ Sufficient data  

📊 Korelasi Matrix:
                HDI_2023  GDP_per_capita  Su

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25188\726895398.py:92: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  ].interpolate(method='linear', limit_direction='both')


In [2]:
# full_panel_pipeline.py
# Run this in one go in Jupyter / VSCode / Colab.
# Produces merged_panel_time_series.csv, EDA outputs, plots, model summary in ./new_analysis_output/

import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from io import StringIO

sns.set(style="whitegrid")

# ---------- CONFIG ----------
BASE_PATH = Path("./Output/merged_clean_panel.csv")   # path to your existing cleaned cross-sectional file
HDI_PATH = Path("./Time-Series-Dataset/hdi-time-series-data.csv")         # optional: full file
SU19_PATH = Path("./Time-Series-Dataset/suicide-rate-2019.csv")
SU20_PATH = Path("./Time-Series-Dataset/suicide-rate-by-country-2020.csv")
SU21_PATH = Path("./Time-Series-Dataset/suicide-rate-by-country-2021.csv")

OUT = Path("./new_analysis_output")
OUT.mkdir(parents=True, exist_ok=True)

# ---------- helper: ISO2 -> ISO3 ----------
def try_import_country_converter():
    try:
        import country_converter as coco
        return coco.CountryConverter()
    except Exception:
        try:
            import pycountry
            return pycountry
        except Exception:
            return None

cc = try_import_country_converter()

def iso2_to_iso3(code):
    if pd.isna(code):
        return np.nan
    code = str(code).strip()
    if not code:
        return np.nan
    # try country_converter
    if cc is None:
        return np.nan
    try:
        # cc may be CountryConverter or pycountry module
        if hasattr(cc, "convert"):
            return cc.convert(names=code, to='ISO3')
        else:
            # pycountry fallback: iso2 -> alpha_3
            c = cc.countries.get(alpha_2=code.upper())
            return c.alpha_3 if c else np.nan
    except Exception:
        return np.nan

# ---------- 1) load base cleaned dataset ----------
if BASE_PATH.exists():
    df_base = pd.read_csv(BASE_PATH)
    print("Loaded base cleaned cross-section:", BASE_PATH)
else:
    # try alt paths or raise helpful message
    alt = Path("/mnt/data/merged_clean_panel.csv")
    if alt.exists():
        df_base = pd.read_csv(alt)
        print("Loaded base cleaned cross-section:", alt)
    else:
        raise FileNotFoundError(f"Base cleaned file not found at {BASE_PATH} or /mnt/data. Please provide merged_clean_panel.csv.")

# ---------- 2) load HDI time series (wide) ----------
if HDI_PATH.exists():
    df_hdi_ts = pd.read_csv(HDI_PATH)
    print("Loaded HDI timeseries from", HDI_PATH)
else:
    # fallback sample (use your full file for real analysis)
    sample = '''"flagCode","country","HumanDevelopmentIndex_2023","HumanDevelopmentIndex_2022","HumanDevelopmentIndex_2021","HumanDevelopmentIndex_2020","HumanDevelopmentIndex_2019","HumanDevelopmentIndex_2010","HumanDevelopmentIndex_HDITierCurrent_txt_YearFree"
"IS","Iceland",0.972,0.959,0.957,0.955,0.958,0.927,"Very High"
"CH","Switzerland",0.97,0.967,0.965,0.957,0.96,0.94,"Very High"
"NO","Norway",0.97,0.966,0.964,0.963,0.961,0.938,"Very High"
"DK","Denmark",0.962,0.952,0.947,0.946,0.946,0.913,"Very High"
"DE","Germany",0.959,0.95,0.948,0.948,0.951,0.929,"Very High"
"SE","Sweden",0.959,0.952,0.949,0.944,0.947,0.91,"Very High"'''
    df_hdi_ts = pd.read_csv(StringIO(sample))
    print("HDI timeseries file not found. Using sample rows (replace with full file).")

# add ISO3 column from flagCode if exists
if 'flagCode' in df_hdi_ts.columns:
    df_hdi_ts['ISO2'] = df_hdi_ts['flagCode'].astype(str).str.replace('"','').str.strip()
    df_hdi_ts['ISO3'] = df_hdi_ts['ISO2'].apply(iso2_to_iso3)
else:
    # attempt to convert country names to ISO3
    if 'country' in df_hdi_ts.columns:
        df_hdi_ts['ISO3'] = df_hdi_ts['country'].apply(iso2_to_iso3)

# ---------- 3) load suicide yearly files and stack ----------
def load_or_sample(path, sample_text):
    if Path(path).exists():
        return pd.read_csv(path)
    else:
        return pd.read_csv(StringIO(sample_text))

su19_sample = '''"flagCode","country","SuicideRateCountries_2019","SuicideRateMaleCountries_2019","SuicideRateFemaleCountries_2019"
"GY","Guyana",40.3,63,17.4
"LT","Lithuania",26.1,45.4,9.6
"KR","South Korea",28.6,40.2,16.9
"RU","Russia",25.1,43.6,9.1
"SR","Suriname",25.4,38.8,11.8
'''
su20_sample = '''"flagCode","country","SuicideRateCountries_2020","SuicideRateMaleCountries_2020","SuicideRateFemaleCountries_2020"
"GL","Greenland",64.6,94.39,31.59
"GY","Guyana",31.71,53.32,10.59
"LT","Lithuania",28.07,50.46,8.93
"KR","South Korea",25.63,36.61,14.56
"RU","Russia",24.25,42.42,8.44
'''
su21_sample = '''"flagCode","country","SuicideRateCountries_2021","SuicideRateMaleCountries_2021","SuicideRateFemaleCountries_2021"
"GL","Greenland",59.62,86.96,29.4
"GY","Guyana",31.26,52.79,10.31
"LT","Lithuania",27.91,49.96,9.07
"KR","South Korea",25.81,36.96,14.58
"RU","Russia",24.1,41.82,8.66
'''
df_su19 = load_or_sample(SU19_PATH, su19_sample)
df_su20 = load_or_sample(SU20_PATH, su20_sample)
df_su21 = load_or_sample(SU21_PATH, su21_sample)

# add ISO3
for d in (df_su19, df_su20, df_su21):
    if 'flagCode' in d.columns:
        d['ISO2'] = d['flagCode'].astype(str).str.replace('"','').str.strip()
        d['ISO3'] = d['ISO2'].apply(iso2_to_iso3)
    else:
        d['ISO3'] = d['country'].apply(iso2_to_iso3)

# helper to create long format per year
def su_to_long(df_su, year):
    # find main suicide column (choose first numeric column not ISO/flag)
    numeric_cols = [c for c in df_su.columns if df_su[c].dtype.kind in 'fi']
    # prefer column containing year
    col = None
    for c in numeric_cols:
        if str(year) in c:
            col = c
            break
    if col is None and numeric_cols:
        col = numeric_cols[0]
    long = df_su[['country','ISO3', col]].rename(columns={col:'Suicide_rate'}).copy()
    long['Year'] = int(year)
    return long.dropna(subset=['ISO3','Suicide_rate'])

su19_long = su_to_long(df_su19, 2019)
su20_long = su_to_long(df_su20, 2020)
su21_long = su_to_long(df_su21, 2021)
df_su_long = pd.concat([su19_long, su20_long, su21_long], ignore_index=True)

# ---------- 4) melt HDI ts wide->long ----------
hdi_cols = [c for c in df_hdi_ts.columns if c.startswith('HumanDevelopmentIndex_') and c.split('_')[-1].isdigit()]
if not hdi_cols:
    # try other heuristics
    hdi_cols = [c for c in df_hdi_ts.columns if 'HumanDevelopment' in c and any(ch.isdigit() for ch in c)]
hdi_long = df_hdi_ts.melt(id_vars=[col for col in ['flagCode','country','ISO3'] if col in df_hdi_ts.columns],
                           value_vars=hdi_cols, var_name='hdi_var', value_name='HDI')
# parse year
hdi_long['Year'] = hdi_long['hdi_var'].str.replace('HumanDevelopmentIndex_','').astype(int)
hdi_long = hdi_long[['country','ISO3','Year','HDI']].dropna(subset=['ISO3','HDI'])

# ---------- 5) Merge HDI long & suicide long -> panel
panel = pd.merge(hdi_long, df_su_long, on=['ISO3','Year'], how='outer', suffixes=('_hdi','_su'))
# bring in base cross-sectional features by ISO3
if 'ISO3' in df_base.columns:
    base_for_merge = df_base.copy()
    # ensure ISO3 exists in base
    if 'ISO3' not in base_for_merge.columns:
        # try converting Country Name to ISO3
        base_for_merge['ISO3'] = base_for_merge['Country Name'].apply(iso2_to_iso3)
    panel = panel.merge(base_for_merge, on='ISO3', how='left', suffixes=('','_base'))
else:
    panel = panel.merge(df_base, left_on='country_hdi', right_on='Country Name', how='left')

# derive Country Name
panel['Country Name'] = panel['country_hdi'].fillna(panel['country_su']).fillna(panel.get('Country Name'))

# reorder
cols_keep = ['Country Name','ISO3','Year','HDI','Suicide_rate','GDP_per_capita','log_GDP_per_capita','income_group_auto','continent','Low_data_quality_flag']
panel = panel[[c for c in cols_keep if c in panel.columns]]

# ---------- 6) Feature engineering & cleaning ----------
# numeric coercion
for c in ['HDI','Suicide_rate','GDP_per_capita','log_GDP_per_capita']:
    if c in panel.columns:
        panel[c] = pd.to_numeric(panel[c], errors='coerce')

panel = panel.sort_values(['ISO3','Year'])
panel['HDI_sq'] = panel['HDI']**2
panel['Suicide_rate_lag1'] = panel.groupby('ISO3')['Suicide_rate'].shift(1)
panel['HDI_lag1'] = panel.groupby('ISO3')['HDI'].shift(1)
panel['Low_data_quality_flag_panel'] = np.where(panel['GDP_per_capita'].isna() | panel['Suicide_rate'].isna(), "⚠️ Low data reliability", "✅ Sufficient data")

# save cleaned panel
out_csv = OUT / "merged_panel_time_series.csv"
panel.to_csv(out_csv, index=False)
print("Saved merged panel:", out_csv)

# ---------- 7) EDA summary ----------
eda_txt = OUT / "eda_panel_summary.txt"
with open(eda_txt, "w", encoding="utf-8") as f:
    f.write("EDA Summary - merged panel\n")
    f.write(f"Rows: {len(panel)}, unique countries: {panel['ISO3'].nunique()}\n\n")
    f.write("Descriptive stats (HDI, Suicide_rate, GDP_per_capita):\n")
    if 'HDI' in panel.columns:
        f.write(panel[['HDI','Suicide_rate','GDP_per_capita']].describe().to_string())
    else:
        f.write("HDI missing\n")
print("Saved EDA summary to", eda_txt)

# ---------- 8) Visualizations ----------
# (A) Time series: sample top average suicide countries
plt.figure(figsize=(10,5))
if 'Suicide_rate' in panel.columns:
    top_iso = panel.groupby('ISO3')['Suicide_rate'].mean().dropna().sort_values(ascending=False).head(6).index.tolist()
    for iso in top_iso:
        sub = panel[panel['ISO3']==iso]
        plt.plot(sub['Year'], sub['Suicide_rate'], marker='o', label=iso)
    plt.legend()
    plt.title("Time series suicide rate (sample top avg countries)")
    plt.xlabel("Year"); plt.ylabel("Suicide rate")
    plt.tight_layout()
    ts_path = OUT / "time_series_top_suicide.png"
    plt.savefig(ts_path); plt.close()
    print("Saved time series plot:", ts_path)
else:
    print("Suicide_rate not present for plotting time series.")

# (B) Scatter HDI vs Suicide (panel combined)
if 'HDI' in panel.columns and 'Suicide_rate' in panel.columns:
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=panel, x='HDI', y='Suicide_rate', hue='Year', palette='viridis', alpha=0.8)
    plt.title("HDI vs Suicide Rate (panel years combined)")
    scatter_path = OUT / "scatter_hdi_panel.png"
    plt.tight_layout()
    plt.savefig(scatter_path); plt.close()
    print("Saved scatter plot:", scatter_path)

# (C) Choropleth latest year (if plotly available)
latest_year = int(panel['Year'].dropna().max()) if 'Year' in panel.columns else None
if latest_year:
    latest = panel[panel['Year']==latest_year]
    try:
        import plotly.express as px
        fig = px.choropleth(latest, locations='ISO3', color='Suicide_rate',
                            hover_name='Country Name', color_continuous_scale='Reds',
                            title=f'Suicide rate ({latest_year})')
        map_html = OUT / f'choropleth_{latest_year}.html'
        fig.write_html(map_html)
        print("Saved choropleth:", map_html)
    except Exception as e:
        print("Plotly not available or choropleth failed:", e)

# ---------- 9) Modeling: pooled OLS with year & continent dummies ----------
mod_df = panel.dropna(subset=['Suicide_rate','HDI','HDI_sq']).copy()
# add log GDP if missing and GDP present
if 'log_GDP_per_capita' not in mod_df.columns and 'GDP_per_capita' in mod_df.columns:
    mod_df['log_GDP_per_capita'] = np.log(mod_df['GDP_per_capita'].where(mod_df['GDP_per_capita']>0, np.nan))

# create continent dummies if present
if 'continent' in mod_df.columns:
    cont_dummies = pd.get_dummies(mod_df['continent'], prefix='cont', drop_first=True)
    mod_df = pd.concat([mod_df, cont_dummies], axis=1)
else:
    cont_dummies = pd.DataFrame()

# year dummies
if 'Year' in mod_df.columns:
    year_dummies = pd.get_dummies(mod_df['Year'].astype(int).astype(str), prefix='Y', drop_first=True)
    mod_df = pd.concat([mod_df, year_dummies], axis=1)
else:
    year_dummies = pd.DataFrame()

# build formula: Suicide_rate ~ HDI + HDI_sq + log_GDP + continent dummies + year dummies
preds = ['HDI','HDI_sq']
if 'log_GDP_per_capita' in mod_df.columns:
    preds.append('log_GDP_per_capita')
preds += list(cont_dummies.columns) + list(year_dummies.columns)
formula = "Suicide_rate ~ " + " + ".join(preds)
print("Model formula:", formula)

try:
    res = smf.ols(formula=formula, data=mod_df).fit(cov_type='HC1')
    model_summary_path = OUT / "panel_model_summary.txt"
    with open(model_summary_path, "w", encoding="utf-8") as f:
        f.write(res.summary().as_text())
    print("Saved model summary to", model_summary_path)
except Exception as e:
    print("Model fitting failed:", e)
    res = None

# ---------- 10) Final save list ----------
files = list(OUT.iterdir())
with open(OUT / "output_files_list.txt", "w", encoding="utf-8") as f:
    for p in files:
        f.write(str(p) + "\n")

print("Pipeline complete. Files in", OUT)
print("\nSample of merged panel (first 10 rows):")
print(panel.head(10).to_string(index=False))

# End of script


nan not found in ISO3


Loaded base cleaned cross-section: Output\merged_clean_panel.csv
Loaded HDI timeseries from Time-Series-Dataset\hdi-time-series-data.csv


nan not found in ISO3
nan not found in ISO3


Saved merged panel: new_analysis_output\merged_panel_time_series.csv
Saved EDA summary to new_analysis_output\eda_panel_summary.txt
Saved time series plot: new_analysis_output\time_series_top_suicide.png
Saved scatter plot: new_analysis_output\scatter_hdi_panel.png
Saved choropleth: new_analysis_output\choropleth_2023.html
Model formula: Suicide_rate ~ HDI + HDI_sq + log_GDP_per_capita + cont_America + cont_Asia + cont_Europe + cont_Oceania + Y_2020 + Y_2021
Saved model summary to new_analysis_output\panel_model_summary.txt
Pipeline complete. Files in new_analysis_output

Sample of merged panel (first 10 rows):
Country Name ISO3  Year   HDI  Suicide_rate  GDP_per_capita  log_GDP_per_capita income_group_auto continent Low_data_quality_flag   HDI_sq  Suicide_rate_lag1  HDI_lag1 Low_data_quality_flag_panel
 Afghanistan  AFG  2010 0.449           NaN      485.022544            6.184195               Low      Asia     ✅ Sufficient data 0.201601                NaN       NaN     ⚠️ Low data r